# Running on Duplin-Onslow

In [18]:
import Pkg
Pkg.activate("./runMetropolizedRecomEnv")
Pkg.instantiate()
Pkg.add("RandomNumbers")
Pkg.add("MetropolizedForestRecom")
using RandomNumbers
using MetropolizedForestRecom

num_dists = 3
rng_seed = 110934571
steps = 100
pop_dev = 0.05
gamma = 0.0 #0 is uniform on forests; 1 is uniform on partitions

pctGraphPath = joinpath("data", "Duplin_Onslow.json")
nodeData = Set(["GEOID", "POPULATION", "AREA", "border_length"]);

# manually read in base_graph so that we can set a unique identifier that 
# combines county and precinct id
base_graph = BaseGraph(pctGraphPath, "POPULATION", inc_node_data=nodeData,
                       area_col="AREA", node_border_col="border_length",
                       edge_perimeter_col="length");

# now that the field "county_and_prec_id" is set, we can use it to create the 
# graph object that we will sample on
graph = Graph(base_graph, "GEOID");

constraints = initialize_constraints()
add_constraint!(constraints, PopulationConstraint(graph, num_dists, pop_dev))
# add_constraint!(constraints, ConstrainDiscontinuousTraversals(graph)) # only needed for multiscale
# add_constraint!(constraints, MaxCoarseNodeSplits(num_dists+1)) # only needed for multiscale

rng = PCG.PCGStateOneseq(UInt64, rng_seed)
partition = Partition(graph, constraints, num_dists; rng=rng);

  Activating project at `~/durhamfunding-1/Redistricting/Archive/runMetropolizedRecomEnv`
   Resolving package versions...
     Project No packages added to or removed from `~/durhamfunding-1/Redistricting/Archive/runMetropolizedRecomEnv/Project.toml`
    Manifest No packages added to or removed from `~/durhamfunding-1/Redistricting/Archive/runMetropolizedRecomEnv/Manifest.toml`
   Resolving package versions...
     Project No packages added to or removed from `~/durhamfunding-1/Redistricting/Archive/runMetropolizedRecomEnv/Project.toml`
    Manifest No packages added to or removed from `~/durhamfunding-1/Redistricting/Archive/runMetropolizedRecomEnv/Manifest.toml`


In [5]:
proposal = build_single_node_flip(constraints)
measure = Measure(gamma, 1.0) # spanning forest measure; first number is 
# exponent on trees, second on linking edges to add elements to the measure
# push_measure!(measure, get_isoperimetric_score, 0.45)

output_file_path = joinpath("output", "DO", 
                            "singleNodeFlip_gamma"*string(gamma)*".jsonl")
writer = Writer(measure, constraints, partition, output_file_path)
push_writer!(writer, get_log_spanning_trees)
push_writer!(writer, get_log_spanning_forests)
push_writer!(writer, get_isoperimetric_scores)

println("startring mcmc")
run_metropolis_hastings!(partition, proposal, measure, steps, rng,
                         writer=writer, output_freq=1);
close_writer(writer)                         

atlasParam["package.version"] = "MetropolizedForestRecom v0.1.2"
startring mcmc


In [15]:
proposal = build_forest_recom2(constraints)
measure = Measure(gamma, 1.0) # spanning forest measure; first number is 
# exponent on trees, second on linking edges to add elements to the measure
# push_measure!(measure, get_isoperimetric_score, 0.45)

output_file_path = joinpath("output", "DO", 
                            "metropolizedRecombination_gamma"*string(gamma)*".jsonl")
writer = Writer(measure, constraints, partition, output_file_path)
push_writer!(writer, get_log_spanning_trees)
push_writer!(writer, get_log_spanning_forests)
push_writer!(writer, get_isoperimetric_scores)

println("startring mcmc")
run_metropolis_hastings!(partition, proposal, measure, steps, rng,
                         writer=writer, output_freq=1);
close_writer(writer)                      

atlasParam["package.version"] = "MetropolizedForestRecom v0.1.2"
startring mcmc


# Running on North Carolina

In [23]:
# TO run this file execute the following in the examples directory: 
#
# julia NC_1level.jl
#
# TO instantiate the environment to run this file execute the following in the 
# examples directory: 
#
#julia --project=./runMetropolizedRecomEnv -e 'using Pkg; Pkg.instantiate()'
#
#

import Pkg
Pkg.activate("./runMetropolizedRecomEnv")
Pkg.instantiate()

using RandomNumbers
using MetropolizedForestRecom

num_dists = 14
rng_seed = 110934571
steps = 100
pop_dev = 0.02
gamma = 0.0 #0 is uniform on forests; 1 is uniform on partitions

pctGraphPath = joinpath("data", "NC_pct21.json")
nodeData = Set(["county", "prec_id", "pop2020cen", "area", "border_length"]);

# manually read in base_graph so that we can set a unique identifier that 
# combines county and precinct id
base_graph = BaseGraph(pctGraphPath, "pop2020cen", inc_node_data=nodeData,
                       area_col="area", node_border_col="border_length",
                       edge_perimeter_col="length");
for ii = 1:length(base_graph.node_attributes)
    county = base_graph.node_attributes[ii]["county"]
    prec_id = base_graph.node_attributes[ii]["prec_id"]
    name = county*"_"*prec_id
    base_graph.node_attributes[ii]["county_and_prec_id"] = name
end

# now that the field "county_and_prec_id" is set, we can use it to create the 
# graph object that we will sample on
graph = MetropolizedForestRecom.Graph(base_graph, "county_and_prec_id");

constraints = initialize_constraints()
add_constraint!(constraints, PopulationConstraint(graph, num_dists, pop_dev))

# only needed for multiscale
# add_constraint!(constraints, ConstrainDiscontinuousTraversals(graph)) 
# add_constraint!(constraints, MaxCoarseNodeSplits(num_dists+1))

rng = PCG.PCGStateOneseq(UInt64, rng_seed)
partition = Partition(graph, constraints, num_dists; rng=rng);

  Activating project at `~/durhamfunding-1/Redistricting/Archive/runMetropolizedRecomEnv`


In [24]:
proposal = build_single_node_flip(constraints)
measure = Measure(gamma, 1.0) # spanning forest measure; first number is 
# exponent on trees, second on linking edges to add elements to the measure
# push_measure!(measure, get_isoperimetric_score, 0.45)

output_file_path = joinpath("output", "NC", 
                            "singleNodeFlip_gamma"*string(gamma)*".jsonl")
writer = Writer(measure, constraints, partition, output_file_path)
push_writer!(writer, get_log_spanning_trees)
push_writer!(writer, get_log_spanning_forests)
push_writer!(writer, get_isoperimetric_scores)

println("startring mcmc")
run_metropolis_hastings!(partition, proposal, measure, steps, rng,
                         writer=writer, output_freq=1);
close_writer(writer)                         

atlasParam["package.version"] = "MetropolizedForestRecom v0.1.2"
startring mcmc


In [25]:
proposal = build_forest_recom2(constraints)
measure = Measure(gamma, 1.0) # spanning forest measure; first number is 
# exponent on trees, second on linking edges to add elements to the measure
# push_measure!(measure, get_isoperimetric_score, 0.45)

output_file_path = joinpath("output", "NC", 
                            "metropolizedRecombination_gamma"*string(gamma)*".jsonl")
writer = Writer(measure, constraints, partition, output_file_path)
push_writer!(writer, get_log_spanning_trees)
push_writer!(writer, get_log_spanning_forests)
push_writer!(writer, get_isoperimetric_scores)

println("startring mcmc")
run_metropolis_hastings!(partition, proposal, measure, steps, rng,
                         writer=writer, output_freq=1);
close_writer(writer) 

atlasParam["package.version"] = "MetropolizedForestRecom v0.1.2"
startring mcmc
